# Linear Elasticity
In this notebook we solve a simple linear elasticity problem in NGSolve.

First, we import the required packages.

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

Next, we generate a mesh for the rectangular domain $\Omega=[0,1]\times [-t/2,t/2]$. We define the boundary $\Gamma_D=\{0\}\times[-t/2,t/2]$ as the Dirichlet boundary where the material is fixed, $u_D=0$, and on the remaining Neumann boundary $\Gamma_N=\partial\Omega\setminus\Gamma_D$ we either might describe a traction force. We name the left and right boundary edges, and restrict the mesh size on the left, where we will prescribe Dirichlet boundary conditions to fix the material.

In [ ]:
def GenerateMesh(t=0.1):
    bar = MoveTo(0,-t/2).Rectangle(1,t).Face()
    bar.edges.Min(X).name="left"
    bar.edges.Max(X).name="right"
    bar.edges.Min(X).maxh=t/6
    mesh = Mesh(OCCGeometry(bar, dim=2).GenerateMesh(maxh=t/3))
    return mesh
mesh = GenerateMesh()
Draw(mesh);

We specify the force as the right-hand side and the material parameters (Young modulus $E$ and Poisson's ratio $\nu$; and the corresponding Lam\'e parameters $\mu$ and $\lambda$). Then we define the linear material law of Hooke, giving as the stress-strain relation
\begin{align*}
\sigma = \mathbb{C}\varepsilon := 2\mu\varepsilon + \lambda \,\mathrm{tr}(\varepsilon)I.
\end{align*}

In [ ]:
force = CF( (0,0.1) )

E = 100    # Young modulus 
nu = 0.2   # Poisson ratio
# Lame parameters
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

# material law of Hooke
def Hooke(eps):
    return 2*mu*eps + lam*Trace(eps)*Id(2)

The problem of linear elasticity reads as follows: For given body force $f$ and traction force $g$ on $\Gamma_N$, find $u\in H^1(\Omega,\mathbb{R}^2)$ such that $u=u_D$ on $\Gamma_D$ and for all $v\in H^1_{\Gamma_D}(\Omega,\mathbb{R}^2)$ there holds
$$ \int_{\Omega}\mathbb{C}\varepsilon(u):\varepsilon(v)\,dx = \int_{\Omega}f\cdot v\,dx + \int_{\Gamma_N} g\cdot v\,ds. $$

We consider either a volume force $f$ or a shear force $g$ acting on the right boundary.

In [ ]:
def Solve(mesh, order=2):
    fes = VectorH1(mesh, order=order, dirichlet="left")
    u,v = fes.TnT()

    a = BilinearForm(fes, symmetric=True)
    a += InnerProduct(Hooke(Sym(Grad(u))),Sym(Grad(v)))*dx
    # alternatively
    #a += (2*mu*InnerProduct(Sym(Grad(u)),Sym(Grad(v)))+lam*div(u)*div(v))*dx

    f = LinearForm(fes)
    f += -force*v*ds("right") # shear force at right boundary
    # f += -force*v*dx # volume force (gravity)

    gfu = GridFunction(fes)
    a.Assemble()
    f.Assemble()

    gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    return gfu

We draw the xx-component of the stress. We see that it is positive where the material is stretched in x-direction and negative when it is compressed.

In [ ]:
mesh = GenerateMesh()
gfu = Solve(mesh)
Draw (Hooke(Sym(Grad(gfu)))[0,0], mesh, deformation=gfu);